In [25]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, DoubleType, StructType, StructField, TimestampType
from pyspark.sql.functions import col, to_timestamp

In [26]:
# create schema
schema = StructType([ 
    StructField("dt",TimestampType(), True), 
    StructField("lat",DoubleType(), True), 
    StructField("lon",DoubleType(), True), 
    StructField("base", StringType(), True), 
  ])

In [27]:
spark = SparkSession.builder.appName('Uber')\
        .config('spark.jars.packages', 'org.postgresql:postgresql:42.2.14')\
        .getOrCreate()


In [28]:
# read data from hdfs 
# path = "hdfs://localhost:9000/raws/raw_*/*" # or "hdfs://localhost:9000/raws/raw_*/*.csv"
path = "hdfs://localhost:9000/raws/" 
df_uber = spark.read.csv(path=path, schema=schema)

# convert dt column to timestamp
# df_uber = df.withColumn("dt",to_timestamp("dt").cast("timestamp"))
df_uber.show(5)
df_uber.printSchema()

+----+-------+--------+-------+
|  dt|    lat|     lon|   base|
+----+-------+--------+-------+
|null| 40.729|-73.9422|B02598"|
|null|40.7476|-73.9871|B02598"|
|null|40.7424|-74.0044|B02598"|
|null| 40.751|-73.9869|B02598"|
|null|40.7406|-73.9902|B02598"|
+----+-------+--------+-------+
only showing top 5 rows

root
 |-- dt: timestamp (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- base: string (nullable = true)



In [29]:
df_uber.count()

1288

In [30]:
from pyspark.ml.feature import VectorAssembler
# Denfine features vector to use for kmeans algorithm
featureCols = ['lat', 'lon']
assembler = VectorAssembler(inputCols=featureCols, outputCol='features')

df_uber2 = assembler.transform(df_uber)
# df_uber2.cache()
df_uber2.show(5)

+----+-------+--------+-------+------------------+
|  dt|    lat|     lon|   base|          features|
+----+-------+--------+-------+------------------+
|null| 40.729|-73.9422|B02598"| [40.729,-73.9422]|
|null|40.7476|-73.9871|B02598"|[40.7476,-73.9871]|
|null|40.7424|-74.0044|B02598"|[40.7424,-74.0044]|
|null| 40.751|-73.9869|B02598"| [40.751,-73.9869]|
|null|40.7406|-73.9902|B02598"|[40.7406,-73.9902]|
+----+-------+--------+-------+------------------+
only showing top 5 rows



In [31]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# setK(20) phân thành 20 cụm
# setFeaturesCol("features") dùng để train
# setPredictionCol("cid") dùng để predict
kmeans = KMeans().setK(20).setFeaturesCol("features").setPredictionCol("cid").setSeed(1)
model = kmeans.fit(df_uber2)

# Shows the result 20 cluster.
centers = model.clusterCenters()
i=0
print("Cluster Centers: ")
for center in centers:
    print(i, center)
    i += 1

Cluster Centers: 
0 [ 40.71570246 -73.95567131]
1 [ 40.69812    -73.92200364]
2 [ 40.69715 -73.58165]
3 [ 40.86407143 -73.9237    ]
4 [ 40.76353537 -73.96920884]
5 [ 40.3495 -74.0667]
6 [ 40.62990667 -73.96354   ]
7 [ 40.740395   -74.00916056]
8 [ 40.74973874 -73.98708333]
9 [ 40.67906456 -73.9870962 ]
10 [ 40.81074857 -73.95151429]
11 [ 40.64623077 -73.78250769]
12 [ 41.015  -73.6916]
13 [ 40.72143617 -73.99729271]
14 [ 41.05735 -74.14275]
15 [ 40.74470625 -73.8257375 ]
16 [ 40.75884667 -73.92368667]
17 [ 41.0019 -74.0405]
18 [ 40.7957 -74.4804]
19 [ 40.78808 -74.12762]


In [32]:
# Evaluate clustering by computing Silhouette score
# evaluator = ClusteringEvaluator(predictionCol='cid', featuresCol='features',
#                                 metricName='silhouette', distanceMeasure='squaredEuclidean')

# silhouette = evaluator.evaluate(predictions)
# print("Silhouette with squared euclidean distance = " + str(silhouette))

In [33]:
#save model
# model.save("E:/PySpark/Uber_Locations/model/uber_location")
# model.write().overwrite().save("E:/PySpark/Uber_Locations/model/uber_location")


In [34]:
# make prediction
df_predicted = model.transform(df_uber2)
df_predicted.show(5)

+----+-------+--------+-------+------------------+---+
|  dt|    lat|     lon|   base|          features|cid|
+----+-------+--------+-------+------------------+---+
|null| 40.729|-73.9422|B02598"| [40.729,-73.9422]|  0|
|null|40.7476|-73.9871|B02598"|[40.7476,-73.9871]|  8|
|null|40.7424|-74.0044|B02598"|[40.7424,-74.0044]|  7|
|null| 40.751|-73.9869|B02598"| [40.751,-73.9869]|  8|
|null|40.7406|-73.9902|B02598"|[40.7406,-73.9902]|  8|
+----+-------+--------+-------+------------------+---+
only showing top 5 rows



In [35]:
from pyspark.sql.functions import split, concat_ws, concat

# add id column = cid + lat + lon
split_lon = split(df_predicted.lon, "\.").getItem(1)
split_lat = split(df_predicted.lat, "\.").getItem(1)
id = concat(split_lat,split_lon) # nối chuỗi
df_uber_id = df_predicted.withColumn("id", concat_ws("_",col("cid"),id)) # add column "id"

# drop feature column
df_uber_locates = df_uber_id.drop(df_uber_id.features)
df_uber_locates.show(5)

+----+-------+--------+-------+---+----------+
|  dt|    lat|     lon|   base|cid|        id|
+----+-------+--------+-------+---+----------+
|null| 40.729|-73.9422|B02598"|  0| 0_7299422|
|null|40.7476|-73.9871|B02598"|  8|8_74769871|
|null|40.7424|-74.0044|B02598"|  7|7_74240044|
|null| 40.751|-73.9869|B02598"|  8| 8_7519869|
|null|40.7406|-73.9902|B02598"|  8|8_74069902|
+----+-------+--------+-------+---+----------+
only showing top 5 rows



In [36]:
# write to hdfs   
# df_uber_locates.write.mode("overwrite").format("csv") \
#                     .option("path", "hdfs://localhost:9000/transformed/") \
#                     .save()

# spark.stop()

df_uber_locates.write \
    .mode("overwrite") \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/Test")\
    .option("dbtable", "TestUber") \
    .option("user", "postgres") \
    .option("password", "Huy12345678") \
    .option("driver", "org.postgresql.Driver") \
    .save()